This first cell makes sure that torchgan is installed. It's primarily here for use with colab, since colab doesn't have torchgan installed by default.

In [1]:
# from torchGAN tutorial

try:
    import torchgan

    print(f"Existing TorchGAN {torchgan.__version__} installation found")
except ImportError:
    import subprocess
    import sys

    subprocess.check_call([sys.executable, "-m", "pip", "install", "torchgan"])
    import torchgan

    print(f"Installed TorchGAN {torchgan.__version__}")

Existing TorchGAN v0.1.0 installation found


In [2]:
import torch
import torch.nn as nn
import torch.backends.cudnn
import torch.utils.data as tdata
from torch.optim import Adam
import torchvision
import torchvision.datasets as dsets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

import torchgan
from torchgan.models import *
from torchgan.losses import *
from torchgan.trainer import Trainer

This cell takes care of creating the dataset. Any PNG or JPEG image files are acceptable, as long as their width and height are the same. The models also require that the dimensions of the image are a perfect power of 2, but that can be overcome by scaling the images.

Another thing to note is that the file path provided as root cannot be the exact folder that holds the images. In order for the path to be recognized properly, you must specify the folder that contains the folder with the images.

For copyright reasons, the images that I am training from are not included with the GitHub repo. If you are interested in learning more, the citations for all of the images are included in the bibliography of the accompanying report.

In [3]:
size = 256

data = ImageFolder(
    root="./data/pictures/preprocessed/complete/",
    transform=transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=.5, std=.5)
    ])
)
print(len(data))

211


In [4]:
dataloader = tdata.DataLoader(data, batch_size=64, shuffle=True)

In [5]:
# adapted from torchgan tutorial

dcgan_network = {
    "generator": {
        "name": DCGANGenerator,
        "args": {
            "encoding_dims": 100,
            "out_size": size,           # this is how many pixels wide and tall the generated images will be
            "out_channels": 3,          # change this to 1 if the images are in black and white
            "step_channels": 32,
            "nonlinearity": nn.LeakyReLU(0.2),
            "last_nonlinearity": nn.Tanh(),
        },
        "optimizer": {"name": Adam, "args": {"lr": 0.0001, "betas": (0.5, 0.999)}},
    },
    "discriminator": {
        "name": DCGANDiscriminator,
        "args": {
            "in_size": size,            # this is how many pixels wide and tall we resized the images to be.
            "in_channels": 3,           # change this to 1 if the images are in black and white
            "step_channels": 32,
            "nonlinearity": nn.LeakyReLU(0.2),
            "last_nonlinearity": nn.LeakyReLU(0.2),
        },
        "optimizer": {"name": Adam, "args": {"lr": 0.0003, "betas": (0.5, 0.999)}},
    },
}

In [6]:
# from torchgan tutorial

minimax_losses = [MinimaxGeneratorLoss(), MinimaxDiscriminatorLoss()]
wgangp_losses = [
    WassersteinGeneratorLoss(),
    WassersteinDiscriminatorLoss(),
    WassersteinGradientPenalty(),
]
lsgan_losses = [LeastSquaresGeneratorLoss(), LeastSquaresDiscriminatorLoss()]

If you're running this on colab, be sure to select a GPU runtime for the best performance.

In [7]:
# more from torchgan tutorial

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    # Use deterministic cudnn algorithms
    torch.backends.cudnn.deterministic = True
    epochs = 10
else:
    device = torch.device("cpu")
    epochs = 5

print("Device: {}".format(device))
print("Epochs: {}".format(epochs))

Device: cpu
Epochs: 5


In [8]:
# from torchgan tutorial

trainer = Trainer(
    dcgan_network,
    lsgan_losses,
    sample_size=64,
    epochs=1000,             # you can override the number of epochs here if you want to try to get better results
    device=device
)

This is the cell that actually does the training. After it finishes, you can run it again to keep training the same model.

In [10]:
trainer(dataloader)

Saving Model at './model/gan0.model'
Epoch 1 Summary
Epoch time duration : 35.821415185928345
generator Mean Gradients : 7.647486985263731
discriminator Mean Gradients : 341.87434565231916
Mean Running Discriminator Loss : 0.13479580695551427
Mean Running Generator Loss : 0.6077807422124453
Generating and Saving Images to ./images/epoch1001_generator.png

Saving Model at './model/gan1.model'
Epoch 2 Summary
Epoch time duration : 35.77073907852173
generator Mean Gradients : 7.639974272064119
discriminator Mean Gradients : 341.78451712187723
Mean Running Discriminator Loss : 0.13473731691819757
Mean Running Generator Loss : 0.6077075987150837
Generating and Saving Images to ./images/epoch1002_generator.png

Saving Model at './model/gan2.model'
Epoch 3 Summary
Epoch time duration : 35.505197286605835
generator Mean Gradients : 7.632463430596368
discriminator Mean Gradients : 341.4944557798314
Mean Running Discriminator Loss : 0.134631456859573
Mean Running Generator Loss : 0.6076270775109

Use this cell instead of the previous one if you want to resume training a model that you saved from a previous session. Be sure to use the correct file path and file name when you call the function.

In [9]:
trainer.load_model("")
trainer(dataloader)